# Upload Raspberry Pi with AWS
- Part 1: Load data from Arduino and store data incorrect format
- Part 2: Connect Raspberry with S3 by boto3

#### Part 1

In [22]:
import serial                   # Python library to read data from Raspberry Pi serial 
from datetime import datetime   # Python library to check datetime
import os                       # Python library to use operating system dependent functionality

The steps to connect and read data of Arduino through USB serial can be found in [this website](https://oscarliang.com/connect-raspberry-pi-and-arduino-usb-cable/).
The serial number in Arduino code should match the serial number of Raspeberry-pi code. The USB interface can be see by this linux command `ls /dev/tty*` 

In [23]:
!ls /dev/tty*

/dev/tty    /dev/tty19	/dev/tty3   /dev/tty40	/dev/tty51  /dev/tty62
/dev/tty0   /dev/tty2	/dev/tty30  /dev/tty41	/dev/tty52  /dev/tty63
/dev/tty1   /dev/tty20	/dev/tty31  /dev/tty42	/dev/tty53  /dev/tty7
/dev/tty10  /dev/tty21	/dev/tty32  /dev/tty43	/dev/tty54  /dev/tty8
/dev/tty11  /dev/tty22	/dev/tty33  /dev/tty44	/dev/tty55  /dev/tty9
/dev/tty12  /dev/tty23	/dev/tty34  /dev/tty45	/dev/tty56  /dev/ttyAMA0
/dev/tty13  /dev/tty24	/dev/tty35  /dev/tty46	/dev/tty57  /dev/ttyprintk
/dev/tty14  /dev/tty25	/dev/tty36  /dev/tty47	/dev/tty58  /dev/ttyUSB1
/dev/tty15  /dev/tty26	/dev/tty37  /dev/tty48	/dev/tty59
/dev/tty16  /dev/tty27	/dev/tty38  /dev/tty49	/dev/tty6
/dev/tty17  /dev/tty28	/dev/tty39  /dev/tty5	/dev/tty60
/dev/tty18  /dev/tty29	/dev/tty4   /dev/tty50	/dev/tty61


In [24]:
try:
    read_serial = serial.Serial("/dev/ttyUSB0", 115200) # 115200 is the serial number in Arduino code
                                                      # dev/ttyUSB0 is the port number optain from ls/dev/tty*
except:
    read_serial = serial.Serial("/dev/ttyUSB1", 115200)

In [35]:
# Create file name which match the date
file_name = datetime.now().strftime("%Y-%m-%d")
file_name = file_name + '.csv'
file_name

'2018-01-11.csv'

In [36]:
# Create columns for the data file
def print_first_line(written_file):
    written_file.write('Date,')
    written_file.write('Patient ID,')
    written_file.write('Red Signal')
    written_file.write('\n')
    return written_file

In [37]:
def check_file_exist(file_name):
    if file_name not in os.listdir(os.curdir):
        # File does not exist, create new file
        written_file = open(file_name, "w")
        written_file = print_first_line(written_file)
    else:
        # File already exist, append new rows
        written_file = open(file_name, "a")
    return written_file

In [ ]:
import time

print("Read data from Serial")
file_signal = check_file_exist(file_name)

start = time.time()
stop = time.time()
try: 
    while (stop - start) < 3:
        data = read_serial.readline()  # Data from serial separate by newline character
        data = data.strip()            # Remove newline character of Arduino IDE 
        file_signal.write(datetime.now().strftime("%H:%M:%S.%f") + ",")    # Record time corresponding to the signal 
        file_signal.write("Vu" + ",")    # Write to column Patient Id
        print(data)
        file_signal.write(data)
        file_signal.write("\n")
        stop = time.time()
except KeyboardInterrupt:
    file_signal.close()
    print("End")

In [30]:
!head 2018-01-10.csv

Date,Patient ID,Red Signal
11:09:45.986920,Vu,13481
11:09:45.988131,Vu,13475
11:09:45.989058,Vu,13482
11:09:45.990266,Vu,13481
11:09:45.991252,Vu,13474
11:09:45.992199,Vu,13474
11:09:45.993140,Vu,13482
11:09:45.994156,Vu,13486
11:09:45.995364,Vu,13480


#### Part 2

In [15]:
import boto3
import botocore

BUCKET_NAME = 'canaria-client-data'

In [16]:
# Print all file inside S3 bucket
session = boto3.Session(profile_name='default') # take secret key in awscli credentials file.
s3_client = session.client('s3')
for key in s3_client.list_objects(Bucket=BUCKET_NAME)['Contents']:
    print(key['Key'])

2018-01-02.csv
DockerToolbox.exe
empty/
empty/2018-01-02.csv
empty/2018-01-10.csv


In [21]:
# Check file exist in S3
for key in s3_client.list_objects(Bucket=BUCKET_NAME)['Contents']:
    if key['Key'] == 'empty/2018-01-02.csv':
        print('Exists')
    else:
        print("Doesn't exist")

Doesn't exist
Doesn't exist
Doesn't exist
Exists


In [31]:
# Upload file to s3
s3_client.upload_file(file_name, BUCKET_NAME, 'empty/'+file_name)

In [37]:
# Download file to Raspberry-pi 
s3_client.download_file(BUCKET_NAME, '2018-01-02.csv', '2018-01-02.csv')

## Finalize
The program will check if the file exists on s3. It the file exists, the program will download and write to the file. If s3 does not have the file, the program will create locally then upload on s3.

In [38]:
def update_file_signal(file_write, file_name, bucket_name):
    file_write.close()                                       # Close the file for uploading
    s3_client.upload_file(file_name, bucket_name, 'empty/' + file_name)             # Uploadfile to s3
    file_write = open(file_write.name, "a")                   # Reopen close file
    return file_write

In [41]:
print("Read data from Serial")
file_signal = check_file_exist(file_name)
counter_minute = 0 

try: 
    while 1:
        now = datetime.now()
        if (now.second) == 59:
            #print("Current Time: %d %d, Past: %d", now.minute, now.second, past)
            counter_minute = counter_minute + 1
        data = read_serial.readline()  # Data from serial separate by newline character
        data = data.strip()            # Remove newline character of Arduino IDE 
        file_signal.write(datetime.now().strftime("%H:%M:%S.%f") + ",")    # Record time corresponding to the signal 
        file_signal.write("Vu" + ",")    # Write to column Patient Id
        file_signal.write(data)
        file_signal.write("\n")
        #if (now.minute == 30) or (now.hour - past == 1):        # Update file every 30 minutes
        if counter_minute == 10:
            file_signal = update_file_signal(file_signal, file_name, BUCKET_NAME)
            print('Uploading File')
            counter_minute = 0
            
except KeyboardInterrupt:
    file_signal.close()
    print("End")

Read data from Serial
Uploading File
Uploading File
Uploading File
Uploading File
Uploading File
Uploading File
Uploading File
Uploading File
Uploading File
Uploading File
Uploading File
Uploading File
End
